In [ ]:
!pip install -U spacy
!pip install jsonlines

     |████████████████████████████████| 6.2 MB 24.7 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 181 kB 56.0 MB/s 
     |████████████████████████████████| 457 kB 46.8 MB/s 
     |████████████████████████████████| 10.1 MB 38.6 MB/s 
     |████████████████████████████████| 653 kB 52.3 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 400.7 MB 6.1 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


# **Imports**

In [ ]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

In [ ]:
#gensim
import gensim
from gensim import corpora

In [ ]:
#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines

In [ ]:
#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#warning
import warnings 
warnings.filterwarnings('ignore')

## **Loading Spacy Model**

In [ ]:
nlp = spacy.load("en_core_web_lg")
skill_pattern_path = "./jz_skill_patterns.jsonl"

# **Entity Ruler**


In [ ]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [ ]:
def unique_skills(x):
    return list(set(x))

def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return unique_skills(subset)

In [ ]:
data = pd.read_csv("jd.csv")
data

,id,category,jd
0,1,cloud engineer,Company is seeking a Data Architect. The indiv...
1,2,cloud engineer,"Cloud Architect Lowell, MA 6+ months Resear..."
2,3,cloud engineer,"Cloud Architect - AWSJob Type: Contract, Contr..."
3,4,cloud engineer,"Senior Software Architect (Perm)Charlestown, M..."
4,5,cloud engineer,"Our client, a well known and highly profitable..."
...,...,...,...
501,502,java developer,Synechron is a recognized leader and expert in...
502,503,java developer,Note: For further information please contact S...
503,504,java developer,"Synechron on behalf of our client, a global le..."
504,505,java developer,Please contactPatrick TrainorCanyon Associates...


In [ ]:
def clean_jds(data):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        data,
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    return review

In [ ]:
#cleaning data
data['cleaned_jd'] = data.jd.apply(lambda x: clean_jds(x))

data['skills'] = data.cleaned_jd.apply(lambda x: get_skills(x))
data

,id,category,jd,cleaned_jd,skills
0,1,cloud engineer,Company is seeking a Data Architect. The indiv...,company seeking data architect individual join...,"[data integrity, documentation, rotation, serv..."
1,2,cloud engineer,"Cloud Architect Lowell, MA 6+ months Resear...",cloud architect lowell 6 month research orient...,"[play, programming language, perl, react, akka..."
2,3,cloud engineer,"Cloud Architect - AWSJob Type: Contract, Contr...",cloud architect awsjob type contract contract ...,"[security, analytics, design, azure, diagram, ..."
3,4,cloud engineer,"Senior Software Architect (Perm)Charlestown, M...",senior software architect perm charlestown job...,"[angular, security, software engineering, docu..."
4,5,cloud engineer,"Our client, a well known and highly profitable...",client well known highly profitable software c...,"[security, design, web service, database, azur..."
...,...,...,...,...,...
501,502,java developer,Synechron is a recognized leader and expert in...,synechron recognized leader expert building bu...,"[design, information management, web service, ..."
502,503,java developer,Note: For further information please contact S...,note information please contact sonal hatila s...,"[angular, oracle, framework, java, spring, ser..."
503,504,java developer,"Synechron on behalf of our client, a global le...",synechron behalf client global leader distribu...,"[security, design, message queue, information ..."
504,505,java developer,Please contactPatrick TrainorCanyon Associates...,please contactpatrick trainorcanyon associates...,"[design, database, testing, java, support, ser..."


In [ ]:
data.to_csv('./jd_with_skills.csv')